In [1]:
%pip install google-cloud-bigquery
%pip install pandas-gbq
%pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "./protean-trilogy-419007-2465e87bee87.json"
)

# Construct a BigQuery client object.
client = bigquery.Client()

In [3]:
import pandas as pd

query = """
SELECT DATE(block_timestamp) as date, COUNT(*) as txns  FROM `bigquery-public-data.goog_blockchain_ethereum_mainnet_us.transactions`
WHERE DATE(block_timestamp) > DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
GROUP BY 1
ORDER BY 1 DESC;

SELECT DATE(block_timestamp) as date, COUNT(*) as txns  FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE DATE(block_timestamp) > DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
GROUP BY 1
ORDER BY 1 DESC;
"""

df = pd.read_gbq(query, project_id='protean-trilogy-419007')

/var/folders/1m/pswlhn1n1gv1c0lymv7l947r0000gn/T/ipykernel_89369/3996777946.py:15: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id='protean-trilogy-419007')


In [4]:
df.tail(10)

,date,txns
0,2024-04-17,193221
1,2024-04-16,1178451
2,2024-04-15,1187906
3,2024-04-14,1133064
4,2024-04-13,1162426
5,2024-04-12,1322052
6,2024-04-11,1213672


Transactions By Address

In [5]:
import pandas as pd

query = """
 SELECT 
        array_to_string(i.addresses, ",") as address,
        i.transaction_hash,
        i.block_number,  
        i.value / 100000000 as value_btc ,
        t.`hash`,
        t.block_timestamp,  
        t.input_count,
        t.output_count,
        t.is_coinbase,
        t.output_value / 100000000 as tx_value_btc , 
        'input' as type
        
    FROM `bigquery-public-data.crypto_bitcoin.inputs` as i     
    INNER JOIN `bigquery-public-data.crypto_bitcoin.transactions` AS t ON t.hash = i.transaction_hash    
    WHERE array_to_string(i.addresses, ',') in UNNEST(["bc1q8yj0herd4r4yxszw3nkfvt53433thk0f5qst4g"])
    
    UNION ALL
    
    SELECT 
        array_to_string(o.addresses, ",") as address,
        o.transaction_hash,
        o.block_number,       
        o.value / 100000000 as value_btc ,
        t.`hash`,
        t.block_timestamp,  
        t.input_count,
        t.output_count,
        t.is_coinbase,
        t.output_value / 100000000 as tx_value_btc, 
        'output' as type
        
    FROM `bigquery-public-data.crypto_bitcoin.outputs` as o     
    INNER JOIN `bigquery-public-data.crypto_bitcoin.transactions` AS t ON t.hash = o.transaction_hash        
    WHERE array_to_string(o.addresses, ',') in UNNEST(["bc1q8yj0herd4r4yxszw3nkfvt53433thk0f5qst4g"])
"""

df = pd.read_gbq(query, project_id="protean-trilogy-419007")
df

/var/folders/1m/pswlhn1n1gv1c0lymv7l947r0000gn/T/ipykernel_89369/206339102.py:41: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id="protean-trilogy-419007")


,address,transaction_hash,block_number,value_btc,hash,block_timestamp,input_count,output_count,is_coinbase,tx_value_btc,type
0,bc1q8yj0herd4r4yxszw3nkfvt53433thk0f5qst4g,9f7605f2464be482cfc825e0889760d645383d76548dc6...,836536,0.000145190,9f7605f2464be482cfc825e0889760d645383d76548dc6...,2024-03-27 18:09:41+00:00,1,32,False,0.483775460,output
1,bc1q8yj0herd4r4yxszw3nkfvt53433thk0f5qst4g,3ca2188e9805310b0cc9d57b2fb800efeb74a6eeb6f616...,835855,78317.034624600,3ca2188e9805310b0cc9d57b2fb800efeb74a6eeb6f616...,2024-03-23 01:55:58+00:00,1,2,False,87051.034624600,output
2,bc1q8yj0herd4r4yxszw3nkfvt53433thk0f5qst4g,477586a8a7f02b54e3562ef42e538e4311228907291860...,836710,0.000015580,477586a8a7f02b54e3562ef42e538e4311228907291860...,2024-03-28 20:11:18+00:00,1,2,False,0.000054520,output
3,bc1q8yj0herd4r4yxszw3nkfvt53433thk0f5qst4g,49961014df6727d1495b0ba486648566e8da0bc12d92eb...,836211,0.000005460,49961014df6727d1495b0ba486648566e8da0bc12d92eb...,2024-03-25 07:03:28+00:00,2,2,False,0.001191350,output


## Top Bitcoin Balances

In [6]:
query = """
WITH double_entry_book AS (
   -- debits
   SELECT array_to_string(inputs.addresses, ",") AS address, inputs.type, -inputs.value AS value
   FROM `bigquery-public-data.crypto_bitcoin.inputs` AS inputs
   UNION ALL
   -- credits
   SELECT array_to_string(outputs.addresses, ",") AS address, outputs.type, outputs.value AS value
   FROM `bigquery-public-data.crypto_bitcoin.outputs` AS outputs
)
SELECT address, type, sum(value) AS balance
FROM double_entry_book
GROUP BY address, type
ORDER BY balance DESC
LIMIT 1000
"""

df = pd.read_gbq(query, project_id="protean-trilogy-419007")
df

/var/folders/1m/pswlhn1n1gv1c0lymv7l947r0000gn/T/ipykernel_89369/1836515136.py:18: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id="protean-trilogy-419007")


,address,type,balance
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,scripthash,24859747193542.000000000
1,bc1qgdjqv0av3q56jvd82tkdjpy7gdp9ut8tlqmgrpmv24...,witness_v0_scripthash,18001007619908.000000000
2,bc1ql49ydapnjafl5t2cp9zqpjwe6pdgmxy98859v2,witness_v0_keyhash,13629481153509.000000000
3,bc1qazcm763858nkj2dj986etajv6wquslv8uxwczt,witness_v0_keyhash,9464330763058.000000000
4,1FeexV6bAHb8ybZjqQMjJrcCrHGW9sb6uF,pubkeyhash,7995726548413.000000000
...,...,...,...
995,35ZFq1z6MsgCptVTZBvLmziaS25LpS72qt,scripthash,200000000000.000000000
996,3EycDjP7xTMWRfMjXPgd2LkBr2Q5ebUxMe,scripthash,200000000000.000000000
997,3PjoeL5wqr2a1HWq1WUg13KsMdZVWopTwS,scripthash,200000000000.000000000
998,3AQxhkcurareVrzqkivDf6sPMYXju8iED5,scripthash,200000000000.000000000
